# Import libraries and data

Dataset was obtained in the capstone project description (direct link [here](https://d3c33hcgiwev3.cloudfront.net/_429455574e396743d399f3093a3cc23b_capstone.zip?Expires=1530403200&Signature=FECzbTVo6TH7aRh7dXXmrASucl~Cy5mlO94P7o0UXygd13S~Afi38FqCD7g9BOLsNExNB0go0aGkYPtodekxCGblpc3I~R8TCtWRrys~2gciwuJLGiRp4CfNtfp08sFvY9NENaRb6WE2H4jFsAo2Z2IbXV~llOJelI3k-9Waj~M_&Key-Pair-Id=APKAJLTNE6QMUY6HBC5A)) and splited manually in separated csv files. They were stored at my personal github account (folder link here) and you can download and paste inside your working directory in order for this notebook to run.

In [3]:
import pandas as pd
import numpy as np

In [62]:
items = pd.read_csv('data/capstone/Capstone Data - Office Products - Items.csv', index_col=0) 
actual_ratings = pd.read_csv('data/capstone/Capstone Data - Office Products - Ratings.csv', index_col=0) 

content_based = pd.read_csv('data/capstone/Capstone Data - Office Products - CBF.csv', index_col=0)
user_user = pd.read_csv('data/capstone/Capstone Data - Office Products - User-User.csv', index_col=0)
item_item = pd.read_csv('data/capstone/Capstone Data - Office Products - Item-Item.csv', index_col=0)
matrix_fact = pd.read_csv('data/capstone/Capstone Data - Office Products - MF.csv', index_col=0)
pers_bias = pd.read_csv('data/capstone/Capstone Data - Office Products - PersBias.csv', index_col=0)

items[['Availability','Price']] = items[['Availability','Price']].apply(lambda col: col.apply(lambda elem: str(elem).replace(',', '.'))).astype(float)

content_based = content_based.apply(lambda col: col.apply(lambda elem: str(elem).replace(',', '.'))).astype(float)
user_user = user_user.apply(lambda col: col.apply(lambda elem: str(elem).replace(',', '.'))).astype(float)
item_item = item_item.apply(lambda col: col.apply(lambda elem: str(elem).replace(',', '.'))).astype(float)
matrix_fact = matrix_fact.apply(lambda col: col.apply(lambda elem: str(elem).replace(',', '.'))).astype(float)
pers_bias = pers_bias.apply(lambda col: col.apply(lambda elem: str(elem).replace(',', '.'))).astype(float)

print('items.shape = ' + str(items.shape))
print('actual_ratings.shape = ' + str(actual_ratings.shape))
print('content_based.shape = ' + str(content_based.shape))
print('user_user.shape = ' + str(user_user.shape))
print('item_item.shape = ' + str(item_item.shape))
print('matrix_fact.shape = ' + str(matrix_fact.shape))
print('pers_bias.shape = ' + str(pers_bias.shape))

actual_ratings.head()

items.shape = (200, 7)
actual_ratings.shape = (200, 100)
content_based.shape = (200, 100)
user_user.shape = (200, 100)
item_item.shape = (200, 100)
matrix_fact.shape = (200, 100)
pers_bias.shape = (200, 100)


,64,65,75,79,83,112,252,271,301,305,...,3411,3430,3524,3533,3625,3902,3991,4047,4342,4462
item,,,,,,,,,,,,,,,,,,,,,
24,NaN,NaN,NaN,NaN,NaN,NaN,4.0,5.0,4.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
35,NaN,NaN,NaN,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
45,NaN,5.0,4.0,5.0,NaN,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
print('Average number of rated items per user')
avg_rated = np.average(actual_ratings.apply(lambda col: np.sum(~np.isnan(col))))
print(str(avg_rated) + ' = ' + str(avg_rated/actual_ratings.shape[0]*100) + '% of all data')


Average number of rated items per user
14.62 = 7.31% of all data


# Class RecommenderEvaluator

In order to become easier to evaluate the metrics, I created a class that receives all the predicted ratings data, including the original ratings data and defined functions to extract all the metrics established in section 1 of the capstone report.

In [214]:
class RecommenderEvaluator:
    
    def __init__(self, items, actual_ratings, content_based, user_user, item_item, matrix_fact, pers_bias):
        
        self.items = items
        self.actual_ratings = actual_ratings
        self.average_rating_per_userid = actual_ratings.apply(lambda row: np.average(row[~np.isnan(row)]))
        
        self.content_based = content_based
        self.user_user = user_user
        self.item_item = item_item
        self.matrix_fact = matrix_fact
        self.pers_bias = pers_bias
        
        self.recommenders_list = [self.content_based, self.user_user, self.item_item, self.matrix_fact,self.pers_bias]
        self.recommenders_list_names = ['content_based', 'user_user', 'item_item', 'matrix_fact','pers_bias']
        
        # Used for item popularity metric.
        # Calculate the 20 most popular items (item which most of the customers bought)
        N_LIM = 20
        perc_users_bought_item = self.actual_ratings.apply(lambda item: np.sum(~np.isnan(item)), axis=0)/actual_ratings.shape[1]
        sort_pop_items = np.argsort(perc_users_bought_item)[::-1]
        self.pop_items = perc_users_bought_item.iloc[sort_pop_items][:N_LIM].index.values.astype(np.int)
    
        
    def get_observed_ratings(self, userId):
        """
        Given user Id, returns all the items he evaluated and their ratings
        :userId: user id
        :return: array of rated items. Index is the item id and value is the item rating
        """
        userId = str(userId)
        filtered_ratings = self.actual_ratings[userId]
        rated_items = filtered_ratings[~np.isnan(filtered_ratings)]
        return rated_items
    
    def get_top_n(self, userId, n):
        userId = str(userId)
        predicted_ratings = dict()
        for recommender, recommender_name in zip(self.recommenders_list,self.recommenders_list_names):
            item_ids = recommender[userId].argsort().sort_values()[:n].index.values
            predicted_ratings[recommender_name] = item_ids
        return predicted_ratings
    
    def rmse(self, userId):
        userId = str(userId)
        observed_ratings = self.get_observed_ratings(userId)
        rmse_list = []
        for recommender in self.recommenders_list:
            predicted_ratings = recommender.loc[observed_ratings.index, userId]
            rmse_list.append(np.sqrt(np.average((predicted_ratings - observed_ratings)**2)))
        rmse_list = pd.Series(rmse_list)
        rmse_list.index = self.recommenders_list_names
        return rmse_list
    
    def nDCG(self, userId):
        ri = self.get_observed_ratings(userId)
        top5 = self.get_top_n(userId,5)

        # 1st step: Given recommendations, transform list into scores (see score transcriptions in the capstone report)
        scores_all = []
        for name, item_list in top5.items():
            scores = np.empty_like(item_list) # initialise 'random' array
            scores[:] = -10                   ###########################
                                                                   # check which items returned by the recommender
            is_already_rated = np.isin(item_list, ri.index.values) # the user already rated. Items users didn't rate
            scores[~is_already_rated] = 0                          # receive score = 0
            for index, score in enumerate(scores):
                if(score != 0):                                    # for each recommended items the user rated
                    if(ri[item_list[index]] < self.average_rating_per_userid[userId] - 1): # score accordingly the report 
                        scores[index] = -1
                    elif((ri[item_list[index]] >= self.average_rating_per_userid[userId] - 1) & 
                         (ri[item_list[index]] < self.average_rating_per_userid[userId] + 0.5)):
                        scores[index] = 1
                    else:
                        scores[index] = 2
            scores_all.append(scores)                              # append all the transformed scores
        scores_all  

        # 2nd step: Given scores, calculate the model's DCG, ideal DCG and then nDCG
        nDCG_all = dict()
        for index_model, scores_model in enumerate(scores_all):   # for each model
            model_DCG = 0                                         # calculate model's DCG
            for index, score in enumerate(scores_model):          #
                index_ = index + 1                                #
                model_DCG = model_DCG + score/np.log2(index_ + 1) #   
            ideal_rank_items = np.sort(scores_model)[::-1]                        # calculate model's ideal DCG
            ideal_rank_DCG = 0                                                    #
            for index, ideal_score in enumerate(ideal_rank_items):                #
                index_ = index + 1                                                #
                ideal_rank_DCG = ideal_rank_DCG + ideal_score/np.log2(index_ + 1) #
            if((ideal_rank_DCG == 0) | (np.abs(ideal_rank_DCG) < np.abs(model_DCG))): # if nDCG is 0 or only negative scores came up
                nDCG = 0 
            else:                                                     # calculate final nDCG when ideal DCG is != 0
                nDCG = model_DCG/ideal_rank_DCG
                                                         
            nDCG_all[self.recommenders_list_names[index_model]] = nDCG # save each model's nDCG in a dict
            # convert it to dataframe
            result_final = pd.DataFrame(nDCG_all, index=range(1)).transpose()
            result_final.columns = ['nDCG']
        return result_final

    def price_diversity(self,userId):

        topn = self.get_top_n(userId,5)

        stats = pd.DataFrame()
        for key, value in topn.items():
            data_filtered = self.items.loc[topn[key]][['Price']].agg(['mean','std']).transpose()
            data_filtered.index = [key]
            stats = stats.append(data_filtered)
        return stats
    
    def availability_diversity(self,userId):

        topn = self.get_top_n(userId,5)

        stats = pd.DataFrame()
        for key, value in topn.items():
            data_filtered = self.items.loc[topn[key]][['Availability']].agg(['mean','std']).transpose()
            data_filtered.index = [key]
            stats = stats.append(data_filtered)
        return stats
    def popularity(self, userId):
        
        topn = self.get_top_n(userId,5)

        recommended = re.get_top_n(userId,10)

        results = {'popularity': []}
        for recommender, recommendations in recommended.items():
            popularity = np.sum(np.isin(recommendations,self.pop_items))
            results['popularity'].append(popularity)
        return pd.DataFrame(results,index = recommenders_list_names)
        

# Test nDCG

In [210]:
userId = '376'
re = RecommenderEvaluator(items, actual_ratings, content_based, user_user, item_item, matrix_fact, pers_bias)
re.nDCG(userId)

,nDCG
content_based,0.0
item_item,1.0
matrix_fact,1.0
pers_bias,1.0
user_user,1.0


# Test Diversity - Price and Availability

In [222]:
USER_ID = '376'
re = RecommenderEvaluator(items, actual_ratings, content_based, user_user, item_item, matrix_fact, pers_bias)

In [223]:
re.price_diversity(USER_ID)

,mean,std
content_based,32.2480,35.466130
user_user,9.7525,7.226508
item_item,6.9600,2.098285
matrix_fact,15.7280,14.594178
pers_bias,9.8900,5.121875


In [224]:
re.availability_diversity(USER_ID)

,mean,std
content_based,0.691080,0.155786
user_user,0.684480,0.159583
item_item,0.568853,0.198758
matrix_fact,0.550033,0.117540
pers_bias,0.588596,0.172630


# Test Popularity

In [225]:
re.popularity(userId)

,popularity
content_based,0
user_user,0
item_item,0
matrix_fact,0
pers_bias,0
